# ML Pipeline Application

This notebook provides examples of how the implemented and deployed ML pipeline can be used to predict calls worthiness.

In [12]:
import os
import json
import pandas as pd
import requests

## Send a POST request to a model

We deployed a model locally for now. The URL to local host can be replaced by the correct URL deending on where the model is deployed.

In [13]:
url = 'http://localhost:5000/predict'

The deployed service requires input data in JSON format.

In [14]:
data_path = os.path.join(os.getcwd(), "../data/data_extracted/bank-additional")
df = pd.read_csv(os.path.join(data_path, "bank-additional.csv"), delimiter=";").head(3)
data = df.to_json()
data = json.loads(data)

Sending the POST request with a sample of 3 clients for whom we want to receive the predicitons:

In [15]:
response = requests.post(url, json=data)

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print(f'Error: {response.text}')

{'Prediction': {'0': 1, '1': 0, '2': 0}, 'Probability-no': {'0': 0.5830667347, '1': 0.7863463619, '2': 0.7396581773}, 'Probability-yes': {'0': 0.4169332653, '1': 0.2136536381, '2': 0.2603418227}}


Making the results more clear to call center employees

In [16]:
calls_list = pd.DataFrame(result)
calls_list['Action'] = ['to call' if x == 1 else 'to skip' for x in calls_list['Prediction']]
calls_list

,Prediction,Probability-no,Probability-yes,Action
0,1,0.583067,0.416933,to call
1,0,0.786346,0.213654,to skip
2,0,0.739658,0.260342,to skip


This final table with predicitons can be sent to call center employees.